In [85]:
import numpy as np
import pandas as pd
import json
import os
from collections import defaultdict
import warnings

## Inspection

In [16]:
indices = np.load("/home/vessl/projects/dynamic-cdfsl/data/cache_cdfsl/ChestX_indices.npy", allow_pickle=True)
train_indices = np.load("/home/vessl/projects/dynamic-cdfsl/data/cache_cdfsl/ChestX_indices_partial_train_0.2_0.npy", allow_pickle=True)
test_indices = np.load("/home/vessl/projects/dynamic-cdfsl/data/cache_cdfsl/ChestX_indices_partial_test_0.2_0.npy", allow_pickle=True)

In [27]:
full = indices.item()[0]

In [28]:
tr = train_indices.item()[0]
te = test_indices.item()[0]

In [41]:
len(full), len(tr) + len(te)

(4215, 4215)

In [31]:
set(full) == set(tr + te)

True

## Adaptation

In [115]:
target = "cub"

In [116]:
index_path = "/home/vessl/projects/dynamic-cdfsl/data/cache_cdfsl/{}_indices.npy".format(target)
map_path = "/home/vessl/projects/dynamic-cdfsl/data/cache_cdfsl/{}_index_path_map.json".format(target)
train_path = "/home/vessl/projects/dynamic-cdfsl/data/cache_cdfsl/{}_indices_partial_train_0.2_0.npy".format(target)
test_path = "/home/vessl/projects/dynamic-cdfsl/data/cache_cdfsl/{}_indices_partial_test_0.2_0.npy".format(target)

train_dest_path = "/home/vessl/projects/dynamic-cdfsl/data/cache_cdfsl_oh/{}_indices_partial_train_0.2_0.npy".format(target)
test_dest_path = "/home/vessl/projects/dynamic-cdfsl/data/cache_cdfsl_oh/{}_indices_partial_test_0.2_0.npy".format(target)

oh_train_path = "/home/vessl/projects/CD-FSL/datasets/split_seed_1/{}_unlabeled_20.csv".format(target)
oh_test_path = "/home/vessl/projects/CD-FSL/datasets/split_seed_1/{}_labeled_80.csv".format(target)

In [117]:
full_index = np.load(index_path, allow_pickle=True).item()
with open(map_path) as f:
    path_map = json.load(f)

def get_path_subset(path):
    d = os.path.basename(os.path.dirname(path))
    b = os.path.basename(path)
    return os.path.join(d, b)

path_map = {
    int(k): get_path_subset(v) for k, v in path_map.items()
}

In [118]:
oh_train = pd.read_csv(oh_train_path, usecols=[1])
oh_test = pd.read_csv(oh_test_path, usecols=[1])

In [119]:
train_paths = list(oh_train.iloc[:,0])
test_paths = list(oh_test.iloc[:,0])

In [120]:
train_index = defaultdict(list)
test_index = defaultdict(list)

In [121]:
for cls, indices in full_index.items():
    for index in indices:
        path = path_map[index]
        if path in train_paths:
            train_index[cls].append(index)
        elif path in test_paths:
            test_index[cls].append(index)
        else:
            warnings.warn("Invalid path,index: {},{}".format(path, index))
            
train_index = dict(train_index)
test_index = dict(test_index)

In [122]:
os.makedirs(os.path.dirname(train_dest_path), exist_ok=True)
os.makedirs(os.path.dirname(test_dest_path), exist_ok=True)
np.save(train_dest_path, train_index)
np.save(test_dest_path, test_index)
print("New split files saved to:")
print(train_dest_path)
print(test_dest_path)

New split files saved to:
/home/vessl/projects/dynamic-cdfsl/data/cache_cdfsl_oh/cub_indices_partial_train_0.2_0.npy
/home/vessl/projects/dynamic-cdfsl/data/cache_cdfsl_oh/cub_indices_partial_test_0.2_0.npy
